In [12]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import root_mean_squared_error

In [13]:
import mlflow
mlflow.set_tracking_uri("$MLFLOW_DB_URI")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/imvaibhavrana/mlops/02-experiment-tracking/$MLFLOW_DB_URI/886638330577656943', creation_time=1757386861713, experiment_id='886638330577656943', last_update_time=1757386861713, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
target = 'duration'
def get_df(parquet_file_path):
    df = pd.read_parquet(parquet_file_path)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df["duration"].dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <=60)]
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']   # creating a new feature
    return df

In [15]:
train_df = get_df("data/yellow_tripdata_2023-01.parquet")
val_df = get_df("data/yellow_tripdata_2023-02.parquet")
train_df = train_df.head(100000)
val_df = val_df.head(100000)

In [16]:
dv = DictVectorizer()
new_feature = ['PU_DO']
train_dict = train_df[new_feature + numerical].to_dict(orient='records')
val_dict = val_df[new_feature + numerical].to_dict(orient='records')

x_train = dv.fit_transform(train_dict)
y_train = train_df[target].values

x_val = dv.transform(val_dict)
y_val = val_df[target].values

In [17]:
with mlflow.start_run():
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag('model', 'Ridge')
    model = Ridge(alpha=alpha, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    print(f"RMSE: {rmse}")
    with open('model.bin','wb') as f_out:
        pickle.dump((dv,model),f_out)
    mlflow.log_artifact('model.bin')

RMSE: 6.235094602309826


In [23]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

In [24]:
dtrain= xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','XGBoost')
        mlflow.log_params(params)
        booster = xgb.train(params=params, dtrain=dtrain, num_boost_round=100, 
                            evals=[(dvalid, 'validation')], early_stopping_rounds=50)
        y_pred = booster.predict(dvalid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 10, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [30]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=4,
    trials=Trials()
)
print(best_result)  

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:8.95848                          
  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

/home/imvaibhavrana/anaconda3/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:07:17] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:8.29705                          
[2]	validation-rmse:7.76207                          
[3]	validation-rmse:7.33311                          
[4]	validation-rmse:6.99035                          
[5]	validation-rmse:6.71937                          
[6]	validation-rmse:6.50496                          
[7]	validation-rmse:6.33834                          
[8]	validation-rmse:6.20624                          
[9]	validation-rmse:6.10192                          
[10]	validation-rmse:6.02138                         
[11]	validation-rmse:5.95714                         
[12]	validation-rmse:5.90469                         
[13]	validation-rmse:5.86446                         
[14]	validation-rmse:5.83497                         
[15]	validation-rmse:5.81128                         
[16]	validation-rmse:5.78959                         
[17]	validation-rmse:5.77317                         
[18]	validation-rmse:5.76072                         
[19]	validation-rmse:5.74890

/home/imvaibhavrana/anaconda3/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:07:25] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:7.85039                                                  
[2]	validation-rmse:7.24727                                                  
[3]	validation-rmse:6.81028                                                  
[4]	validation-rmse:6.49979                                                  
[5]	validation-rmse:6.27813                                                  
[6]	validation-rmse:6.12189                                                  
[7]	validation-rmse:6.00987                                                  
[8]	validation-rmse:5.93060                                                  
[9]	validation-rmse:5.87550                                                  
[10]	validation-rmse:5.83244                                                 
[11]	validation-rmse:5.80202                                                 
[12]	validation-rmse:5.77804                                                 
[13]	validation-rmse:5.76111                                    

/home/imvaibhavrana/anaconda3/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:07:32] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.37444                                                   
[1]	validation-rmse:5.84361                                                   
[2]	validation-rmse:5.76082                                                   
[3]	validation-rmse:5.73081                                                   
[4]	validation-rmse:5.73202                                                   
[5]	validation-rmse:5.73223                                                   
[6]	validation-rmse:5.73122                                                   
[7]	validation-rmse:5.72856                                                   
[8]	validation-rmse:5.72561                                                   
[9]	validation-rmse:5.71676                                                   
[10]	validation-rmse:5.71183                                                  
[11]	validation-rmse:5.71005                                                  
[12]	validation-rmse:5.70847                        

/home/imvaibhavrana/anaconda3/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:07:40] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:5.87001                                                   
[2]	validation-rmse:5.77468                                                   
[3]	validation-rmse:5.74648                                                   
[4]	validation-rmse:5.73859                                                   
[5]	validation-rmse:5.73841                                                   
[6]	validation-rmse:5.73426                                                   
[7]	validation-rmse:5.73711                                                   
[8]	validation-rmse:5.73435                                                   
[9]	validation-rmse:5.73152                                                   
[10]	validation-rmse:5.72829                                                  
[11]	validation-rmse:5.73057                                                  
[12]	validation-rmse:5.72827                                                  
[13]	validation-rmse:5.72676                        